In [1]:
import os 
import pandas as pd 
import teradata
import numpy as np
import logging # getting annoying logging message when creating table, turning it off 
logging.getLogger().setLevel(logging.CRITICAL)
import getpass



In [2]:
user = 'grumpy_statistician'
auth ='LDAP'
passw = getpass.getpass()

········


### Connect to Server

In [3]:
udaExec = teradata.UdaExec(appName="test", version="2.0", logConsole=False)
session = udaExec.connect(method="odbc", system='',
                          username=user, password=passw, authentication=auth,
                          driver="Teradata Database ODBC Driver")


### Create Volatile table

In [4]:
session.execute("""
CREATE VOLATILE TABLE TMP_MEM AS (
SELECT 
*
FROM YOUR_DB.YOUR_TABLE_NAME
)WITH DATA ON COMMIT PRESERVE ROWS;
""")

### Execute multiple volatile tables from one file
If we were reading the sql file we would use:
with open ('Your_Script_Name.sql','r') as sql_file:
        test = sql_file3.read().
For this example I am just going to write in some example volatile tables, separate them into a list and then iterate through that list to execute them.


In [5]:
# I just repeated the same code over and over since it doesn't really matter
test = """
CREATE VOLATILE TABLE TMP_MEM1 AS (
SELECT 
*
FROM YOUR_DB.YOUR_TABLE_NAME
)WITH DATA ON COMMIT PRESERVE ROWS;

CREATE VOLATILE TABLE TMP_MEM2 AS (
SELECT 
*
FROM YOUR_DB.YOUR_TABLE_NAME
)WITH DATA ON COMMIT PRESERVE ROWS;

CREATE VOLATILE TABLE TMP_MEM3 AS (
SELECT 
*
FROM YOUR_DB.YOUR_TABLE_NAME
)WITH DATA ON COMMIT PRESERVE ROWS;


"""

Now we can split and create our list of sql queries


In [6]:
sqlCommands = test.split(';')

In [8]:
for i in sqlCommands:
    try:
        session.execute(i)
    except Exception as e:
        print(e)

Done. You can verify by using pd.read_sql() later.

### Insert Records into the database
Executemany can be utilized to add multiple records into a database, there is a limit to the number of records you can add in batch, but this can be avoided.
Let's create a table and try to insert some records into it (the try and except were added to drop the table incase it was already loaded).

In [35]:
try:
    session.execute("""
    CREATE TABLE YOUR_DB.YOUR_TABLENAME (
        thing1 VARCHAR(6),
        thing2 NUMBER(6))
              ; """)
    print('complete.')
except:
    print('dropping...')
    session.execute("""
    DROP TABLE YOUR_DB.YOUR_TABLENAME 
              ; """)  
    session.execute("""
    CREATE TABLE YOUR_DB.YOUR_TABLENAME (
        thing1 VARCHAR(6),
        thing2 NUMBER(6))
              ; """)
    print('complete.')

dropping...
done.


Read in one of the volatile tables to a dataframe

In [37]:
df =pd.read_sql("""
SELECT 
*
FROM TMP_MEM3

""",session)

Create list from records and load into a table using executemany. There are two ways to do this, the first option is suitable for smaller files less than 10k. The second will be for larger files.

##### Option 1

In [40]:
data = [tuple(x) for x in df.to_records(index=False)]

In [45]:
session.executemany("""
INSERT INTO YOUR_DB.YOUR_TABLENAME
( thing1,thing2)
VALUES
(?,?)
""",data,batch=True)

Read results and check

In [50]:
df_table = pd.read_sql("""
SELECT
*
FROM YOUR_DB.YOUR_TABLENAME

""",session)

##### Option 2

First, we should create the table

In [83]:
try:
    session.execute("""
    CREATE TABLE YOUR_DB.YOUR_TABLENAME (
       thing1 varchar(20))
              ; """)
    print('complete.')
except:
    print('dropping...')
    session.execute("""
    DROP TABLE YOUR_DB.YOUR_TABLENAME
              ; """)  
    session.execute("""
    CREATE TABLE YOUR_DB.YOUR_TABLENAME (
       thing1 varchar(20))
              ; """)
    print('complete.')

complete.


Now let's bring in a table with more rows

In [84]:
df2= pd.read_sql('''

SELECT 
distinct
thing1
FROM YOUR_DB.YOUR_TABLENAME
SAMPLE 20000

''',session)

Split into sub-arrays for batch loading and assign to new variable

In [85]:
df_len = len(df2.index)
chunk = ceil(df_len/10000)
chunks_df = np.array_split(df,chunk)

In [86]:
for i,_ in enumerate(chunks_df):
    data = [tuple(x) for x in chunks_df[i].to_records(index=False)]
    session.executemany("""
INSERT INTO YOUR_DB.YOUR_TABLENAME
(thing1)
VALUES
(?)
""",data,batch=True)

In [87]:
df_table2 = pd.read_sql("""
SELECT
*
FROM YOUR_DB.YOUR_TABLENAME

""",session)

Be sure to drop the tables before closing this out!

In [91]:
try:
    session.execute("""
    DROP TABLE YOUR_DB.YOUR_TABLENAME;
    """)
except:
    print('already dropped!')
try:
    session.execute("""
    DROP TABLE YOUR_DB.YOUR_TABLENAME;
    """)
except: 
    print('already dropped!')

already dropped!
already dropped!
